In [22]:
import os
from pathlib import Path
import re
import zipfile
import json

In [23]:
def add_to_recognition_json(filename):
    pattern = r"([^_]+)_"  # Regex pattern to match the group before the first "_"
    match = re.search(pattern, filename)

    if match:
        extracted_group = match.group(1)
        numbers_as_string = re.findall(r'\d+',extracted_group)
        numbers = list(map(int,numbers_as_string))
        letters = re.findall(r'\D+',extracted_group)
        if letters[0] == 'n':
            if letters[1] == 'w':
               numbers[1] = - numbers[1]
        if letters[0] == 's':
            numbers[0] = - numbers[0]
            if letters[1] == 'w':
                numbers[1] = - numbers[1]
        bbox = [(numbers[0],numbers[1]),(numbers[0] + 20, numbers[1]),(numbers[0] + 20, numbers[1] + 30), (numbers[0], numbers[1] + 30)]
        recog_json[filename] = bbox



def unzip_and_rename_files(folder_path):
    filelist = os.listdir(os.path.join(Path.cwd().parent, folder_path))
    for file_name in filelist:
        if file_name.endswith('.zip'):
            zip_file_path = os.path.join(os.path.join(Path.cwd().parent, folder_path), file_name)
            destination_path = os.path.join(Path.cwd().parent, destination_folder) # Remove the '.zip' extension

            # Create the destination folder if it doesn't exist
            if not os.path.exists(destination_path):
                os.makedirs(destination_path)

            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                for tif_file_name in zip_ref.namelist():
                    if tif_file_name.endswith(extension):
                        zip_ref.extract(tif_file_name, destination_path)
                        add_to_recognition_json(tif_file_name)

# Example usage
recog_json = {}
file_source_path = 'DEM_Data'
destination_folder = 'raw_DEM'
extension = 'mea075.tif'
unzip_and_rename_files(file_source_path)
with open('recog.json', 'w') as json_file:
    json.dump(recog_json, json_file)